In [1]:
import abc
import math
from fractions import Fraction

# Introduction
The goal of this notebook is to create some classes that represent different types of real numbers for which some exact arithmetic can be performed, and then compute continued fractions for these numbers. Floating point arithmetic will not be used anywhere.

The purpose of this notebook was for me to get some practice with OOP in Python by doing some fun calculations.

## Note on libraries
The libraries `math` and `fractions` imported above are not strictly necessary. 

The only function from the math library that is used below is the `gcd` function, which I could define myself. The function `math.floor` is invoked repeatedly below, but these uses all actually call code from `__floor__` methods in the classes defined in this notebook.

I originally defined a `RationalNumber` class in this notebook that would have removed the need for the `Fraction` class to be imported, but I decided to cut it since they were the least interesting class of numbers here. The `Fraction` class or something equivalent is needed so that exact arithmetic with fractions can be performed in several places.

# The abstract base class
First I define an abstract base class `ExactRealNumber`. This specifies the methods that all concrete subclasses must instantiate, and uses them to construct other methods.

Concrete subclasses of exact real numbers are required to support just the following operations, where x is an instance of the subclass and n is an integer:
* x+n
* x==n
* x>n
* 1/x

From those operations, it will be possible to calculate the continued fraction for any instance of the class.

In [2]:
class ExactRealNumber(abc.ABC):
    @abc.abstractmethod
    def __add__(self, other):
        """Must be defined when other is an integer."""

    @abc.abstractmethod
    def __gt__(self, other):
        """Must be defined when other is an integer."""
        
    @abc.abstractmethod
    def __eq__(self, other):
        """Must be defined when other is an integer."""

    @abc.abstractmethod 
    def __rtruediv__(self, other):
        """Must be defined when other is 1, if self is non-zero."""
        
    def __sub__(self, other):
        return self + (-1)*other
    
    def __radd__(self, other):
        return self+other
    
    def __rmul__(self, other):
        #Not all subclasses will have multiplication, but if they do, it will be commutative.
        return self*other
    
    def __ne__(self,other):
        return not self == other
    
    def __le__(self,other):
        return not self > other
    
    def __lt__(self, other):
        return (not self > other) and (not self == other)
    
    def __ge__(self, other):
        return (self > other) or (self == other)
    
    def __floor__(self):
        #This method will be overridden in some cases.
        num=self
        floor = 0
        if num > 0:
            while num >= floor+1:
                floor += 1
        elif num < 0:
            while num < floor:
                floor += -1
        return floor

Here is the polymorphic continued fraction function that can be applied to any concrete instance of a subclass:

In [3]:
def continued_fraction_terms(x, n):
    #Returns the first n terms of the continued fraction,
    #or the complete continued fraction, if it has less than n terms.
    i=0
    rn = x
    cflist=[]
    while i < n:
        fl=math.floor(rn)
        cflist.append(fl)
        rn=rn -fl
        if rn == 0:
            break
        else:
            rn=1/rn
            i += 1
    return cflist

This can also be applied to numbers from the existing `Fraction` class, which allows for exact rational arithmetic.

For example:

In [4]:
continued_fraction_terms(Fraction(233,144),10)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

It will also be useful to define a custom exception type to deal with initialization errors for the different classes.

In [5]:
class InitializationError(Exception):
    def __init__(self, message):
        self.message=message

# Logarithms of rational numbers
One kind of number for which continued fractions can be exactly calculated are logarithms of rational numbers with rational bases. This set of numbers is not even closed under addition or multiplcation, but using the identities 1/log_b(a)=log_a(b) and log_b(a)+k=log_b(a\*b^k) the required operations can be defined.

Another distinction between this class and the others defined in this notebook is that these numbers are all rational or transcendental, by the [Gelfond-Schneider theorem](https://en.wikipedia.org/wiki/Gelfond%E2%80%93Schneider_theorem). All the other numbers in this notebook will be algebraic.

In [6]:
class LogNumber(ExactRealNumber):
    def __init__(self,base,a):
        self.base=Fraction(base)
        self.a=Fraction(a)
        if self.a <= 0:
            raise InitializationError('The argument of the logarithm must be positive, but the given argument was {}.'.format(str(a)))
        if self.base == 1 or self.base < 0:
            raise InitializationError('The base of the logarithm should be positive and not equal to 1, but the given base was {}.'.format(str(base)))

        
    def __add__(self,other):
        if isinstance(other,int):
            return LogNumber(self.base,self.a*(self.base)**other)
        else:
            return NotImplemented
        
    def __gt__(self,other):
        if other==0:
            if self.base > 1:
                return self.a > 1
            elif self.base < 1:
                return self.a < 1
        else:
            return (self-other) > 0
        
    def __rtruediv__(self,other):
        if self != 0:
            if isinstance(other,int):
                return LogNumber(self.a,self.base**other)
            else:
                return NotImplemented
        else:
            raise ZeroDivisionError()
    
    def __mul__(self,other):
        if isinstance(other, int):
            return LogNumber(self.base,self.a**other)
        else:
            return NotImplemented
        
    def __eq__(self,other):
        if isinstance(other,int):
            return (self.base)**other==self.a
        else:
            return NotImplemented
    
    def __str__(self):
        return "log_"+str(self.base)+"("+str(self.a)+")"

Now we can use the methods in the base class and the functions defined above to calculate the continued fraction for log base 2 of 3.

In [7]:
continued_fraction_terms(LogNumber(2,3),10)

[1, 1, 1, 2, 2, 3, 1, 5, 2, 23]

Note that some of these log numbers are rational. For example, log_4(8)=3/2. In this case, everything will still work:

In [8]:
continued_fraction_terms(LogNumber(4,8),10)

[1, 2]

# Quadratic numbers
The next class of numbers I'll look at here are solutions to quadratic equations with rational coefficients. By the quadratic formula, these can all be represented in the form (a+b\sqrt{d})/c, where a, b, c, and d are integers.

These numbers are known to have continued fractions that are exactly periodic, so this class has a special method to return the non-repeating part and repeating part of the continued fraction. This algorithm uses hashing to check if a quadratic number has already been seen during the process, so the `__hash__` method will be defined for this class.

In [9]:
class QuadraticNumber(ExactRealNumber):
    #Represents a quadratic number of the form (a+b\sqrt{d})/c, with a, b, c, and d integers.
    def __init__(self, d, a=0, b=1, c=1, normalized=False):
        #Set normalized=True if you know that c > 0 and a,b,c have no common divisor.
        #This way you can save the time of normalizing the number.
        if c==0:
            raise ZeroDivisionError('QuadraticNumber'+str((a,b,c,d)))
        if d<=1:
            #There should also be an initialization error if d is not square free,
            #but I'm skipping that to streamline the code.
            raise InitializationError('The radicand must be greater than 1, but the given radicand was {}.'.format(str(d)))
        for x in [a,b,c,d]:
            if not isinstance(x, int):
                raise InitializationError('The four arguments must be integers.')
        self.a=a 
        self.b=b
        self.c=c
        self.d=d
        if not normalized:
            self.normalize()

    def normalize(self):
        #These numbers need to be normalized so that hashing will work.
        commondiv=math.gcd(self.a,math.gcd(self.b,self.c))
        self.a=self.a//commondiv
        self.b=self.b//commondiv
        self.c=self.c//commondiv
        if self.c <0:
            self.a=-self.a
            self.b=-self.b
            self.c=-self.c
            
    def __gt__(self,other):
        if other==0:
            if self.a == 0:
                return self.b > 0
            if self.b == 0:
                return self.a > 0
            if self.a > 0 and self.b > 0:
                return True
            if self.a < 0 and self.b < 0:
                return False
            if self.a > 0 and self.b < 0:
                return self.a**2 > self.d*self.b**2
            if self.a < 0 and self.b > 0:
                return self.a**2 < self.d*self.b**2
        else:
            return (self - other)>0
    
    def __add__(self, other):
        if isinstance(other, int):
            return QuadraticNumber(self.d,self.a+self.c*other,self.b,self.c,normalized=True)
        elif isinstance(other, QuadraticNumber):
            if self.d != other.d:
                return NotImplemented
            else:
                return QuadraticNumber(self.d, self.a*other.c+self.c*other.a,self.b*other.c+other.b*self.c,self.c*other.c)
            
    def __rtruediv__(self, other):
        #Defined only when other is 1, though it could be defined more generally.
        if self != 0:
            if other==1:
                return QuadraticNumber(self.d, self.c*self.a, -1*self.c*self.b, self.a**2-self.d*self.b**2)
            else:
                return NotImplemented
        else:
            raise ZeroDivisionError()

    def __eq__(self,other):
        if isinstance(other, int):
            return self.b==0 and self.a==self.c*other
        if isinstance(other, QuadraticNumber):
            return self.a == other.a and self.b == other.b and self.c==other.c and self.d == other.d
        else:
            return NotImplemented
        
    def __hash__(self):     
        return hash((self.a,self.b,self.c,self.d))

    def __str__(self):
        bpart=""
        cpart=""
        dpart=""
        if self.a==0 and self.b==0:
            return "0"
        if self.c!=1:
            cpart="/"+str(self.c)
        else:
            cpart=""
        dpart="sqrt("+str(self.d)+")"
        if self.b==0:
            return str(self.a)+cpart
        if self.a==0:
            if self.b==1:
                return dpart+cpart
            elif self.b==-1:
                return "-"+dpart+cpart
            else:
                return str(self.b)+"*"+dpart+cpart
        if self.b == 1:
            bpart = "+"
        elif self.b == -1:
            sgnbpart = "-"
        elif self.b >0:
            inner = "+"+str(self.b)+"*"
        elif self.b <0:
            inner = "-"+str(-1*self.b)+"*"
        return "("+str(self.a)+bpart+dpart+")"+cpart
    
    def repeating_continued_fraction(self):
        #Returns an ordered pair whose first element is the non-repeating part
        #and whose second element is the repeating part.
        qn=self
        cflist=[]
        cfdict={}
        i=0
        while qn not in cfdict:
            fl=math.floor(qn)
            cflist.append(fl)
            cfdict[qn]=i
            i+=1
            qn=qn - fl
            qn=1/qn
        start_of_rep = cfdict[qn]
        return cflist[:start_of_rep],cflist[start_of_rep:]

First, let's use this code to find the continued fraction of sqrt(2).

In [10]:
sqrt2=QuadraticNumber(2)
sqrt2.repeating_continued_fraction()

([1], [2])

The code successfully reached the well known answer that the continued fraction for the square root of 2 is 1 followed by 2 repeating.

Another notable example is φ=(1+sqrt(5))/2, the golden ratio, whose continued fraction has all terms equal to 1, as this function finds:

In [11]:
phi=QuadraticNumber(5,1,1,2)
phi.repeating_continued_fraction()

([], [1])

Here are some examples with longer periods:

In [12]:
qns=[QuadraticNumber(62),QuadraticNumber(62,0,1,2),QuadraticNumber(2,0,1,62)]
for q in qns:
    print(q)
    print(q.repeating_continued_fraction())
    print()

sqrt(62)
([7], [1, 6, 1, 14])

sqrt(62)/2
([3], [1, 14, 1, 6])

sqrt(2)/62
([0, 43], [1, 5, 3, 1, 1, 1, 4, 1, 1, 11, 1, 42, 1, 11, 1, 1, 4, 1, 1, 1, 3, 5, 1, 86])



# General irrational algebraic numbers
Beyond quadratic numbers, it is possible to do exact arithmetic with general real algebraic numbers. You can represent a real algebraic number by giving the following data: A rational polynomial of which the number is a root, and lower and upper bounds for an interval in which that number is the only root of the polynomial.
## A note on generality
This notebook's methods only cover irrational numbers, because that allows some code to be streamlined, and I am only interested in using this class to calculate continued fractions for irrational algebraic numbers.

Also, right now only addition with integers is defined. It is possible to add or multiply two algebraic represented as polynomials and intervals, but this is complicated and not needed for calculating continued fractions.

## Polynomials
To begin with, I need a class for polynomials with rational coefficients.

In [13]:
class Polynomial():
    def __init__(self,coeff_list):
        self.coeff_list=[Fraction(c) for c in coeff_list]

        #Cut off extraneous zero high-degree terms, always leaving a constant term.
        while self.coeff_list[-1]==0 and len(self.coeff_list)>1:
            self.coeff_list.pop()
        self.degree=len(self.coeff_list)-1
        #Adjust so that the degree of the 0 polynomial is -1, following a standard convention. 
        if self.degree==0 and self.coeff_list[0]==0:
            self.degree=-1
            
    def evaluate(self,a):
        #Returns f(a).
        val=0
        for c in reversed(self.coeff_list):
            val=c+a*val
        return val
    
    def deriv(self):
        #Returns the derivative of f.
        if self.degree < 1:
            return 0
        cl=[]
        for i, c in enumerate(self.coeff_list[1:]):
            cl.append((i+1)*c)
        return Polynomial(cl)

    def shift(self,a):
        #Returns the polynomial f(x+a)
        if self.degree < 1:
            return self
        poly=self
        scl=[]
        fact=1
        for i in range(self.degree+1):
            scl.append(poly.evaluate(a)/fact)
            poly=poly.deriv()
            fact=fact*(i+1)
        return Polynomial(scl)
    
    def __str__(self):
        #Could be improved, but communicates all information.
        strlist = [str(c)+"*x^"+str(i) for i, c in enumerate(self.coeff_list)]
        return "+".join(strlist)

## The class IrrationalAlgebraicNumber
With the polynomial class, I can define this class by representing an algebraic number as a root of a polynomial in a specified interval.

Many of the methods, including a `__floor__` method overriding the one in the base class, work by using a bisection method to refine the bounds on the number.

### Current limitations
Right now this class only works properly if the polynomial f and bounds a, b used to initialize an instance satisfy the following three conditions:
* f has a unique root in the interval (a,b).
* This root is irrational.
* f(a) < 0 and f(b) > 0.

The initialization method only checks the third condition. Using some polynomial algorithms, it would be possible to also check the first two conditions and make the third unnecessary. For simplicity none of this is done here, but it may be done in a future version of this notebook.

Note that these conditions are easy to satisfy for nth roots of whole numbers: If d > 1, the polynomial x^n-d satisfies these conditions with bounds 1, d.

In [14]:
class IrrationalAlgebraicNumber(ExactRealNumber):
    #Here are the constraints the input data needs to satisfy for everything to work:
    ##r is irrational
    ##r is the unique root of f in (a,b), where a and b are the lower and upper bounds
    ##f(a)<0 and f(b)>0
    #Only the third constraint is actually checked.
    def __init__(self, f, lower_bound, upper_bound):
        if not lower_bound < upper_bound:
            raise InitializationError('The lower bound should be less than the upper bound.')
        if f.evaluate(lower_bound) >= 0 or f.evaluate(upper_bound) <= 0:
            raise InitializationError('The polynomial must be negative at the lower bound and positive at the upper bound for the current methods to work. This restriction may be removed in the future.')
        self.poly=f
        self.lower_bound = Fraction(lower_bound)
        self.upper_bound = Fraction(upper_bound)

    def __rtruediv__(self,other):
        while (self.lower_bound < 0 and self.upper_bound > 0) or self.lower_bound==0 or self.upper_bound==0:
        #Because the root is irrational it is non-zero so this bisection is guaranteed to terminate.
            self.bisect_bounds()
        #After the above loop, the bounds are guaranteed to both be positive or both be negative
        #so that their reciprocals can bound the reciprocal root.
        n= self.poly.degree
        new_coeff_list=list(reversed(self.poly.coeff_list))
        if self.lower_bound > 0 or n%2==0:
            #In this case the polynomial needs to be negated so it has the correct signs at the bounds.
            new_coeff_list=[-c for c in new_coeff_list]
        return IrrationalAlgebraicNumber(Polynomial(new_coeff_list), 1/self.upper_bound, 1/self.lower_bound)
    
    def bisect_bounds(self):
        m = (self.lower_bound + self.upper_bound)/2
        val = self.poly.evaluate(m)
        if val > 0:
            self.upper_bound = m
        elif val < 0:
            self.lower_bound = m
                
    def __str__(self):
        return "The unique root of "+str(self.poly)+" between "+str(self.lower_bound)+" and "+str(self.upper_bound)+"."
        
    def __add__(self,other):
        if isinstance(other, int):
            newpoly = self.poly.shift(-other)
            return IrrationalAlgebraicNumber(newpoly, self.lower_bound+other, self.upper_bound+other)
        else:
            return NotImplemented
        
    def __eq__(self,other):
        if isinstance(other,int) or isinstance(other,Fraction):
            return False #Since the number is irrational.
        else:
            return NotImplemented
        
    def __gt__(self,other):
        if isinstance(other,int) or isinstance(other,Fraction):
            while other >= self.lower_bound and other < self.upper_bound:
            #Guaranteed to terminate since the number is irrational.
                self.bisect_bounds()
        else:
            return NotImplemented

    def __floor__(self):
        while math.floor(self.lower_bound)+1 != math.ceil(self.upper_bound):
            #Bisect until the lower and upper bounds are in the interval [floor(r), ceiling(r)].
            self.bisect_bounds()
        return math.floor(self.lower_bound)

Now, let's look at some examples:

In [15]:
def nthroot(n,d):
    #If d is an integer > 1 and n is a positive integer,
    #returns an IrrationalAlgebraicNumber representing the nth root of d.
    coef_list=[0 for i in range(n+1)]
    coef_list[0]=-d
    coef_list[-1]=1
    return IrrationalAlgebraicNumber(Polynomial(coef_list),1,d)

In [16]:
continued_fraction_terms(nthroot(3,2),20)

[1, 3, 1, 5, 1, 1, 4, 1, 1, 8, 1, 14, 1, 10, 2, 1, 4, 12, 2, 3]

In [17]:
continued_fraction_terms(nthroot(4,2),20)

[1, 5, 3, 1, 1, 40, 5, 1, 1, 25, 2, 3, 1, 6, 2, 1, 1, 2, 1, 2]

In [18]:
continued_fraction_terms(nthroot(5,2),20)

[1, 6, 1, 2, 1, 1, 1, 3, 25, 1, 4, 3, 3, 7, 52, 1, 2, 3, 2, 15]

In [19]:
continued_fraction_terms(nthroot(100,2),20)

[1, 143, 1, 3, 2, 1, 6, 3, 1, 4, 1, 3, 10, 1, 1, 45, 62, 1, 1, 2]